# Line Following Robot CoppeliaSim

## Setup

Make sure to have the server side running in CoppeliaSim: 
- in a child script of a CoppeliaSim scene, add following command to be executed just once, at simulation start:

```
simRemoteApi.start(19999)
```
then start simulation, and run this program.

IMPORTANT: for each successful call to simxStart, there
should be a corresponding call to simxFinish at the end!

In [ ]:
try:
    import sim
except:
    print ('--------------------------------------------------------------')
    print ('"sim.py" could not be imported. This means very probably that')
    print ('either "sim.py" or the remoteApi library could not be found.')
    print ('Make sure both are in the same folder as this file,')
    print ('or appropriately adjust the file "sim.py"')
    print ('--------------------------------------------------------------')
    print ('')

import time
import numpy as np

We have instructed CoppeliaSim to start a communication server on port 19999. The following statement is used to establish communication

In [ ]:
sim.simxFinish(-1) # just in case, close all opened connections
clientID=sim.simxStart('127.0.0.1',19999,True,True,5000,5) # Connect to CoppeliaSim in synchronous

if clientID!=-1:
    print ('Connected to remote API server')
else:
    print ('Failed connecting to remote API server')  

## Robot Initialization

On the first part of the script, we need to get the joint objects, sensor objects and set their operation modes.

- Object handles are fetched using the blocking mode
- The first call to simxReadVisionSensor() should be "streaming" and the consecutive calls should be "buffer" mode

In [ ]:
# Motor handles
motor = [-1, -1, -1, 1]  # motor[0] motor[1] are left, motor[2] motor[3] are right

# Sensor handles
sensorHandle = [-1,-1,-1,-1,-1]

# Now try to retrieve handles in a blocking fashion (i.e. a service call):

if clientID!=-1:

    # ---------------------Initialize Motor--------------------- #
    
    err, motor[0] =sim.simxGetObjectHandle(clientID, '/Motor_FL', sim.simx_opmode_blocking)
    if(err != 0):
        print("Error getting motor 1")
    err, motor[1] =sim.simxGetObjectHandle(clientID, '/Motor_BL', sim.simx_opmode_blocking)
    if(err != 0):
        print("Error getting motor 2")
    err, motor[2] =sim.simxGetObjectHandle(clientID, '/Motor_FR', sim.simx_opmode_blocking)
    if(err != 0):
        print("Error getting motor 3")
    err, motor[3] =sim.simxGetObjectHandle(clientID, '/Motor_BR', sim.simx_opmode_blocking)
    if(err != 0):
        print("Error getting motor 4")
    
    # ---------------------Initialize Sensors--------------------- #
    
    err, sensorHandle[0] = sim.simxGetObjectHandle(clientID, '/Vis_0', sim.simx_opmode_blocking)
    if(err != 0):
        print("Error_cam_0")    
    err, sensorHandle[1] = sim.simxGetObjectHandle(clientID, '/Vis_1', sim.simx_opmode_blocking)
    if(err != 0):
        print("Error_cam_1")    
    err, sensorHandle[2] = sim.simxGetObjectHandle(clientID, '/Vis_2', sim.simx_opmode_blocking)
    if(err != 0):
        print("Error_cam_2")    
    err, sensorHandle[3] = sim.simxGetObjectHandle(clientID, '/Vis_3', sim.simx_opmode_blocking)
    if(err != 0):
        print("Error_cam_3")    
    err, sensorHandle[4] = sim.simxGetObjectHandle(clientID, '/Vis_4', sim.simx_opmode_blocking)
    if(err != 0):
        print("Error_cam_4")
     
    # ---------------------Enable Streaming--------------------- #
    
    err, state, _ = sim.simxReadVisionSensor(clientID, sensorHandle[0], sim.simx_opmode_streaming) 
    err, state, _ = sim.simxReadVisionSensor(clientID, sensorHandle[1], sim.simx_opmode_streaming)
    err, state, _ = sim.simxReadVisionSensor(clientID, sensorHandle[2], sim.simx_opmode_streaming) 
    err, state, _ = sim.simxReadVisionSensor(clientID, sensorHandle[3], sim.simx_opmode_streaming) 
    err, state, _ = sim.simxReadVisionSensor(clientID, sensorHandle[4], sim.simx_opmode_streaming)
    
    print("Done Initialization")

    time.sleep(2)
    
else:
    print ('Failed connecting to remote API server')

## Control parameters

In this part, we set up some functions to control the robot

- a function for reading sensor easily
- a function for controlling motor speeds easily

In [ ]:
def readSensor():
    
    sensorReading = np.array([0, 0, 0, 0, 0]).astype(float)
    
    # Try to retrieve the streamed data from 5 sensors in a loop
    # the 11th value is the average pixel intensity of the vision sensor
    # We threshold this data to detect line
    
    for i in range(5):
        err, state, value = sim.simxReadVisionSensor(clientID, sensorHandle[i], sim.simx_opmode_buffer) 
        sensorReading[i] = value[0][11]<0.5 # if pixel intensity low, sensor is on black line, otherwise white
    
    return sensorReading 

In [ ]:
def motorControl(leftV, rightV):

    # Two of the left motors use left velocity, two of the right motors use right velocity
    sim.simxSetJointTargetVelocity(clientID, motor[0], leftV, sim.simx_opmode_oneshot)
    sim.simxSetJointTargetVelocity(clientID, motor[1], leftV, sim.simx_opmode_oneshot)
    sim.simxSetJointTargetVelocity(clientID, motor[2], rightV, sim.simx_opmode_oneshot)
    sim.simxSetJointTargetVelocity(clientID, motor[3], rightV, sim.simx_opmode_oneshot)

## Control Loop

- Get readings from sensor
- Complete necessary logical/mathematical operations to detect line
- Send speed commands to motor

In [ ]:
minSpeed = 50*np.pi/180
maxSpeed = 300*np.pi/180

speed = (minSpeed + maxSpeed)/2

leftV = 0
rightV = 0


if clientID!=-1:
    print ("Running LFR Routine")
     
    startTime = time.time()
    
    # Loop for 100 seconds
    while time.time()-startTime < 100: 
        
        # ----------------------------------- Get sensor data ----------------------------------- #
        sensorReading = readSensor()
        # print(sensorReading)
        
        # ----------------------------------- Analyze data/Decision ----------------------------------- #

        leftV = speed
        rightV = speed
        
        if(sensorReading[0]):
            leftV=0.03*speed
        if(sensorReading[4]):
            rightV=0.03*speed
        
        # ----------------------------------- Send motor command ----------------------------------- #

        motorControl(leftV, rightV)
        
else:
    print ('Failed connecting to remote API server')

## Terminate simulation

After simulation ends, we need to properly disconnect from the simulation

In [ ]:
sim.simxFinish(clientID)

    
print ('Program ended')